In [3]:
# Sempre rode isso para começar o Notebook
# De modo resumido, isso faz com que o notebook possa acessar o Django

import sys
sys.path.append("../../")

import os
import django

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serase_projeto.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [4]:
# Digite seu código aqui!

In [16]:
# Testando values_list no PadraoMovimentacao

from serase_app.models import PadraoMovimentacao
from django.db.models import F

lista = PadraoMovimentacao.objects.values("id", "descricao", "periodo", "dia_cobranca", "data_inicio", "data_fim", valor_padrao=F("valor"), categoria=F("categoria__nome"), tipo=F("receita_despesa"))
lista = list(lista)
lista

SyntaxError: invalid syntax (<ipython-input-16-701879c80dad>, line 6)